In [2]:
import pandas as pd
import numpy as np
from collections import Counter

# === 文件路径 ===
main_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\2_regression_original.xlsx"
smiles_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\smiles.xlsx"
output_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_with_weighted_features.xlsx"

# === 读取数据 ===
df_main = pd.read_excel(main_path)
df_smiles = pd.read_excel(smiles_path)

# === 预处理：统一 material_name 为索引 ===
df_smiles = df_smiles.copy()
df_smiles["material_name"] = df_smiles["material_name"].str.strip()
df_smiles.set_index("material_name", inplace=True)

# === 数值型特征列 ===
num_features = ["Tg", "mw_estimate", "viscosity_mPas"]

# === 将 Tg 转为 float（去掉非数字） ===
def parse_float(x):
    try:
        return float(str(x).strip().replace("°C", "").replace("℃", ""))
    except:
        return np.nan

df_smiles["Tg"] = df_smiles["Tg"].apply(parse_float)

# === 类别型特征列 ===
cat_features = ["grade_code", "Brand"]

# === 所有成分列（从 df_main 中自动识别） ===
ingredient_columns = [col for col in df_main.columns if col in df_smiles.index]

# === 函数：计算每一行配方的加权特征 ===
def weighted_feature(row, feature_name, feature_type="num"):
    weighted_values = []
    weights = []
    for col in ingredient_columns:
        amount = row.get(col, 0)
        if pd.notna(amount) and amount > 0:
            try:
                val = df_smiles.at[col, feature_name]
            except KeyError:
                continue
            if pd.notna(val):
                weights.append(amount)
                weighted_values.append(val)
    if not weighted_values or sum(weights) == 0:
        return np.nan

    if feature_type == "num":
        arr = np.array(weighted_values, dtype=float)
        return np.average(arr, weights=weights)
    elif feature_type == "cat":
        counts = Counter()
        for val, w in zip(weighted_values, weights):
            counts[val] += w
        return counts.most_common(1)[0][0]
    else:
        return np.nan

# === 执行加权融合 ===
for feat in num_features:
    df_main[f"weighted_{feat}"] = df_main.apply(lambda row: weighted_feature(row, feat, "num"), axis=1)

for feat in cat_features:
    df_main[f"weighted_{feat}"] = df_main.apply(lambda row: weighted_feature(row, feat, "cat"), axis=1)

# === 保存结果 ===
df_main.to_excel(output_path, index=False)
print("✅ 加权融合完成，已保存至：", output_path)


✅ 加权融合完成，已保存至： D:\ML-3DPrinting-Project\data\7.9_datasets\regression_with_weighted_features.xlsx


In [2]:
import pandas as pd

# 1. 读取两个文件
path_chemberta = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_with_chemberta_rdkit.xlsx"
path_mordred = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_with_mordred.xlsx"

df_rdkit = pd.read_excel(path_chemberta)
df_mordred = pd.read_excel(path_mordred)

# 2. 提取 rdkit_0 至 rdkit_127 列
rdkit_cols = [f"rdkit_{i}" for i in range(128)]
df_rdkit_subset = df_rdkit[rdkit_cols].copy()

# 3. 将 rdkit 列嫁接到 mordred 文件末尾
df_combined = pd.concat([df_mordred, df_rdkit_subset], axis=1)

# 4. 保存结果
save_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_all_chembert_rdkit_mordred.xlsx"
df_combined.to_excel(save_path, index=False)

print("✅ 合并完成，文件保存为：", save_path)


✅ 合并完成，文件保存为： D:\ML-3DPrinting-Project\data\7.9_datasets\regression_all_chembert_rdkit_mordred.xlsx


In [1]:
import pandas as pd

# 读取包含 weighted_Tg 的文件
tg_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_with_Tg.xlsx"
df_tg = pd.read_excel(tg_path)

# 读取目标文件
target_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_all_chembert_rdkit_mordred.xlsx"
df_target = pd.read_excel(target_path)

# 检查行数是否一致（必须对得上）
assert len(df_tg) == len(df_target), "两个文件的行数不一致，无法直接拼接！"

# 提取 Tg 列并拼接
df_target['weighted_Tg'] = df_tg['weighted_Tg'].values

# 保存新文件
output_path = r"D:\ML-3DPrinting-Project\data\7.9_datasets\regression_all_final.xlsx"
df_target.to_excel(output_path, index=False)

print("✅ 拼接完成，新文件保存为 regression_all_with_Tg.xlsx")


✅ 拼接完成，新文件保存为 regression_all_with_Tg.xlsx
